# High Performance Computing with

<center><img src="images/julia_logo.svg" width="300" style="border:0"></center>

<p style="text-align: center;">Emmanuel Jeanvoine - University of Lausanne</p>


## Agenda

- Why Julia?
- Vectorization
- Shared memory/multi-core
- Distributed memory/multi-node
- GPU



## Why Julia?



### Why Julia has been created?

From the creators (see https://julialang.org/blog/2012/02/why-we-created-julia/)

“We want the **speed of C** with the dynamism of Ruby. We want something **as usable for general programming as Python**, **as easy for statistics as R**, as natural for string processing as Perl, **as powerful for linear algebra as Matlab**, as good at gluing programs together as the shell. Something that is **dirt simple to learn**, yet keeps the most serious hackers happy. We want it **interactive** and we want it **compiled**.”



### Julia is nice!

- High level programming language
- Easy to write (similar syntax to Python)
- REPL (read-eval-print loop), compatible with Jupyter Notebook
- Efficient: Python << Julia < C


→ [Julia in the wild](https://juliacomputing.com/case-studies/)




### Compilation pipeline

1. type inference generates typed code (result available with @code_typed)
1. typed code is compiled to LLVM IR (result available with @code_llvm)
1. LLVM IR is transformed by LLVM to fast native code (result available with @code_native)
1. native code gets executed

### Compilation pipeline (under the hood)

In [1]:
function add(a, b)
  return a + b
end

@code_typed add(4, 3)
#@code_llvm add(2, 3)
#@code_native add(2, 3)

CodeInfo(
1 ─ %1 = Base.add_int(a, b)::Int64
└──      return %1
) => Int64

### Performance comparison with other languages

<center><img src="images/benchmarks.svg" width="750" style="border:0"></center>

See https://julialang.org/benchmarks/

## Vectorization


### Vectorization of loops using `@simd`macro

- Julia permits to easily improve execution of loops if:
  - each iteration of the loop is independant from the others
  - the loop body doesn't contain branch or function calls
  - the number of iterations is obvious
  - bounds checking is disabled
- Result of vectorization can be checked using `@code_native` macro


### `@simd` macro

In [2]:
using BenchmarkTools

function vecsum(x, y)
    s = zero(eltype(x))
    for i = eachindex(x)
        @inbounds s += x[i]*y[i]
    end
    return s
end

function vecsum_simd(x, y)
    s = zero(eltype(x))
    @simd for i = eachindex(x)
        @inbounds s += x[i] * y[i]
    end
    return s
end

x = rand(Float32, 10000)
y = rand(Float32, 10000)
@btime s = vecsum($x, $y);
@btime s = vecsum_simd($x, $y);

  15.460 μs (0 allocations: 0 bytes)
  1.047 μs (0 allocations: 0 bytes)


### Explicit vectorization

- `SIMD.jl` package
  - `Vec` type (support of `Bool Int{8,16,32,64,128} UInt{8,16,32,64,128} Float{16,32,64}`)
  - vector operations
    - `+ - * / % ^ ! ~ & | $ << >> >>> == != < <= > >=`
    - `abs cbrt ceil copysign cos div exp exp10 exp2 flipsign floor fma inv isfinite isinf isnan issubnormal log log10 log2 muladd rem round sign signbit sin sqrt trunc vifelse`
  - reduction operations: `all any maximum minimum sum prod`
  - more details on https://juliapackages.com/p/simd


### Explicit vectorization


In [3]:
using BenchmarkTools
using SIMD

println("Naive sum")
a = [1.0, 2.0 , 3.0, 4.0]
@btime sum($a);

println("Vectorized sum")
b = Vec{4, Float64}((1.0, 2.0, 3.0, 4.0))
@btime sum($b);

Naive sum
  5.821 ns (0 allocations: 0 bytes)
Vectorized sum
  3.912 ns (0 allocations: 0 bytes)


### Static arrays

- Optimized arrays when
  - size is small (less than 100 elements)
  - size is known in advance


In [4]:
using BenchmarkTools
using StaticArrays

a = [1, 2, 3, 4]
b = SVector(1, 2, 3, 4)

println("Using regular array")
@btime $a*$a';

println("Using static array")
@btime $b*$b';

Using regular array
  86.983 ns (1 allocation: 192 bytes)
Using static array
  10.837 ns (0 allocations: 0 bytes)


## Shared memory/multi-core


### Linear algebra

- Julia is based on OpenBLAS for linear algebra operations
- Most of the operations are optimized and benefits from multi-core architectures
- Extensive documentation: https://docs.julialang.org/en/v1/stdlib/LinearAlgebra/


### Linear algebra - matrix multiplication example


In [5]:
using BenchmarkTools
using LinearAlgebra

A = rand(5000,5000)
B = rand(5000,5000)
C = similar(A)

println("Matrix multiplication with 1 thread")
BLAS.set_num_threads(1)
@btime mul!($C, $A, $B);

println("Matrix multiplication with 2 threads")
BLAS.set_num_threads(2)
@btime mul!($C, $A, $B);

println("Matrix multiplication with 4 threads")
BLAS.set_num_threads(4)
@btime mul!($C, $A, $B);

Matrix multiplication with 1 thread
  6.810 s (0 allocations: 0 bytes)
Matrix multiplication with 2 threads
  3.459 s (0 allocations: 0 bytes)
Matrix multiplication with 4 threads
  1.957 s (0 allocations: 0 bytes)


> Function that end with `!` are in-place operations (output buffer must be pre-allocated)


### Asynchronous programming with tasks

- Tasks are a convenient way to run asynchronously several sequences of instuctions
- Illusion of parallel sections but no benefit taken from multicores
- Switching between tasks has no cost for the system
- Suitable for sections with heavy I/O operations (disk or network)


### Asynchronous programming with tasks - `@async`/`@sync` macros


In [6]:
println("Sleep without tasks")
@time for i in 1:5
    sleep(1)
end

println("Sleep with tasks")
@time @sync for i in 1:5
    @async sleep(1)
end

println("Compute without tasks")
@time for i in 1:100
    sin.(rand(1000, 1000))
end

println("Compute with tasks")
@time @sync for i in 1:100
    @async sin.(rand(1000, 1000))
end

Sleep without tasks
  5.010470 seconds (278 allocations: 13.945 KiB)
Sleep with tasks
  1.045527 seconds (13.67 k allocations: 781.392 KiB, 3.38% compilation time)
Compute without tasks
  2.112985 seconds (400 allocations: 1.490 GiB, 7.53% gc time)
Compute with tasks
  1.953798 seconds (22.54 k allocations: 1.491 GiB, 5.56% gc time, 1.67% compilation time)


### Threads with `@threads` macro

In [7]:
using Base.Threads

@threads for i in 1:nthreads()
  println(threadid())
end

1
4
3
2


3 important components:
- `@threads`: run a loop over threads
- `nthreads()`: get the number of available threads
- `threadid()`: get the identifier of the current thread

### Choosing the number of threads in Julia

- Set `JULIA_NUM_THREADS` in the environment
  - `export JULIA_NUM_THREADS=4`
- Use `-t` parameter of Julia interpreter
  - `julia -t 4 myscript.jl`
- Use `BLAS.set_num_threads()` function from `LinearAlgebra` package


### Basic computation using threads


In [8]:
using Base.Threads

println("Compute without threads")
@time for i in 1:4
    sin.(rand(5000, 5000))
end

println("Compute with threads")
@time @threads for i in 1:4
    sin.(rand(5000, 5000))
end

Compute without threads
  2.541996 seconds (16 allocations: 1.490 GiB, 11.35% gc time)
Compute with threads
  0.672597 seconds (64.12 k allocations: 1.493 GiB, 13.31% compilation time)


### Requirements for practical sessions

- Install the required packages from a GPU node
```bash
julia -e 'import Pkg; Pkg.add.(["Distributed","SIMD",
                                "LoopVectorization",
                                "Folds","BenchmarkTools",
                                "LinearAlgebra",
                                "StaticArrays","Random",
                                "CSV","YAML",
                                "Plots","CUDA","ClusterManagers"])'
```

### Practical session 1: Basic parallelization

- Go to `array_processing` folder and modify `array_processing.jl` in order to use multi-threading
- Study the scalability of your code when you vary the number of threads from 1 to 8
  - `julia array_processing.jl`
  - `julia -t 2 array_processing.jl`
  - `julia -t 4 array_processing.jl`
  - `julia -t 6 array_processing.jl`
  - `julia -t 8 array_processing.jl`

### ⚠️ Be careful to race conditions  

In [9]:
using Base.Threads

function sum_serial(n)
  acc = 0
  for i in 1:n
    acc += 1
  end
  return acc
end

function sum_threads(n)
  acc = 0
  @threads for i in 1:n
    acc += 1
  end
  return acc
end

println("Sum serial " * string(sum_serial(1_000_000)))
println("Sum threads " * string(sum_threads(1_000_000)))

Sum serial 1000000
Sum threads 250000


### To avoid race conditions

- [Atomic operations](https://docs.julialang.org/en/v1/manual/multi-threading/#Atomic-Operations)
- `Base.Threads.Mutex()` with `lock()`/`unlock()`
- `Base.Semaphore(n)` with `acquire()`/`release()`
- Local variables + reduction

### Practical session 2: Pi estimation with Riemann sum method

<center><img src="images/pi_computing.svg" width="300" style="border:0"></center>

$$\int_{a}^b f(x)dx = \sum\limits_{i=0}^n f(\frac{x_i + x_{i-1}}{2}) \Delta x$$

### Practical session 2: Pi estimation with Riemann sum method

- Go to `pi` folder and complete `pi_parallel` function from `pi.jl` in order to use multi-threading
- Pay attention to
  - race conditions
  - the type of range variables (should be Integer or it ruins performance)
- Study the scalability with 1, 4, 8 cores

### ⚠️ Over-subscription

- Some libraries like LinearAlgebra manage their own threads
- Adding another layer of threads on the top such libraries could ruin performance


In [10]:
using Base.Threads
using BenchmarkTools

function matmul_serial(x)
  first_num = zeros(length(x))
  for i in eachindex(x)
    @inbounds first_num[i] = (x[i]'*x[i])[1]
  end
  return first_num
end

function matmul_thread(x)
  first_num = zeros(length(x))
  @threads for i in eachindex(x)
    @inbounds first_num[i] = (x[i]'*x[i])[1]
  end
  return first_num
end

m = [rand(1000, 1000) for _ in 1:100]
@btime matmul_serial($m);
@btime matmul_thread($m);


  1.576 s (201 allocations: 762.94 MiB)
  1.627 s (234 allocations: 762.95 MiB)


### LoopVectorization.jl package

- `@turbo` macro can be used to vectorize loops
- `@tturbo` macro adds the multi-threading level

→ [More information on LoopVectorization.jl](https://juliasimd.github.io/LoopVectorization.jl/stable/)

### `@tturbo` example: Pi computation using integrals

In [11]:
using BenchmarkTools
using LoopVectorization

function pi(num_steps)
  step = 1.0 / num_steps
  sum = 0.0
  for i in 0:num_steps - 1
    sum += 4.0 / (1.0 + ((i + 0.5) * step)^2)
  end
  return sum
end

function pi_tturbo(num_steps)
  step = 1.0 / num_steps
  sum = 0.0
  @tturbo for i in 0:num_steps - 1
    sum += 4.0 / (1.0 + ((i + 0.5) * step)^2)
  end
  return sum
end

@btime pi(1_000_000_000);
@btime pi_tturbo(1_000_000_000);

  1.676 s (0 allocations: 0 bytes)
  198.694 ms (0 allocations: 0 bytes)


### `Folds.jl`

- `Folds.jl` provides multi-threaded versions of `count`, `findall`, `map`, `reduce`, …

→ [More information on Folds.jl](https://juliafolds.github.io/Folds.jl/stable/)

### `Folds.jl` example: Pi computation using integrals

In [12]:
using BenchmarkTools
using Folds

function pi(num_steps)
  step = 1.0 / num_steps
  sum = reduce(+, (4.0 / (1.0 + ((i + 0.5) * step)^2) for i in 0:num_steps - 1))
  return sum
end

function pi_folds(num_steps)
  step = 1.0 / num_steps
  sum = Folds.reduce(+, (4.0 / (1.0 + ((i + 0.5) * step)^2) for i in 0:num_steps - 1))
  return sum
end

@btime pi(1_000_000_000);
@btime pi_folds(1_000_000_000);

  1.656 s (0 allocations: 0 bytes)
  457.090 ms (29 allocations: 1.62 KiB)


## Distributed memory/multi-node


### Master-worker paradigm

`Distributed` module offers a master-worker way of computing over several processes (local or remote)
- `procs()`: list available workers
- `@everywhere`: execute a function over all workers (useful to import packages)
- `@spawnat X`: execute a function on worker X
- `fetch()`: wait for remote execution termination
- `@distributed`: parallel for loop (split all elements of a loop over workers, reduction is possible)
- `pmap()`: parallel map


### Master-worker paradigm

In [13]:
using Distributed
addprocs(4)
nprocs()

5

In [14]:
fetch(@spawnat :any println("I'm working alone"))

      From worker 2:	I'm working alone


In [15]:
@everywhere println("Hello !")

Hello !
      From worker 4:	Hello !
      From worker 3:	Hello !
      From worker 5:	Hello !
      From worker 2:	Hello !


In [16]:
handler = @spawnat 5 rand(1000,1000)^2
result = fetch(handler);

### Distributed PI computation

In [17]:
using Distributed
using BenchmarkTools

@everywhere function loop(istart, iend, step)
  sum = 0.0
  for i in istart:iend
    sum += 4.0 / (1.0 + ((i + 0.5) * step)^2)
  end
  return sum
end

function pi_serial(num_steps)
  step = 1.0 / num_steps
  return loop(0, num_steps - 1, step) / num_steps
end

function pi_pmap(num_steps)
  step = 1.0 / num_steps
  chunk_size = num_steps ÷ (nprocs() - 1)
  vstart = collect(range(0, num_steps - chunk_size, step=chunk_size))
  vend = vstart .+ (chunk_size - 1)
  vstep = fill(step, nprocs() - 1)
  r = sum(pmap((istart, iend, step)->loop(istart, iend, step), vstart, vend, vstep))
  return r / num_steps
end

function pi_distributed(num_steps)
  step = 1.0 / num_steps
  chunk_size = num_steps ÷ (nprocs() - 1)
  r = @distributed (+) for i in 0:(nprocs() - 2)
    istart = i * chunk_size
    iend = istart + (chunk_size - 1)
    loop(istart, iend, step)
  end
  return r / num_steps
end

println("Pi serial")
#println(pi_serial(100_000_000))
@btime pi_serial(1_000_000_000);

println("Pi pmap")
#println(pi_pmap(100_000_000))
@btime pi_pmap(1_000_000_000);

println("Pi distributed")
#println(pi_distributed(1_000_000_000))
@btime pi_distributed(1_000_000_000);

Pi serial
  1.641 s (0 allocations: 0 bytes)
Pi pmap
  416.292 ms (345 allocations: 14.23 KiB)
Pi distributed
  417.017 ms (329 allocations: 14.05 KiB)


### ClusterManager: integration with Slurm

```julia
using Distributed
using ClusterManagers
using BenchmarkTools

addprocs_slurm(parse(Int, ENV["SLURM_NTASKS"]))

@everywhere function loop(istart, iend, step)
  sum = 0.0
  for i in istart:iend
    sum += 4.0 / (1.0 + ((i + 0.5) * step)^2)
  end
  return sum
end

function pi_pmap(num_steps)
  step = 1.0 / num_steps
  chunk_size = num_steps ÷ (nprocs() - 1)
  vstart = collect(range(0, num_steps - chunk_size, step=chunk_size))
  vend = vstart .+ (chunk_size - 1)
  vstep = fill(step, nprocs() - 1)
  r = sum(pmap((istart, iend, step)->loop(istart, iend, step), vstart, vend, vstep))
  return r / num_steps
end

println("Pi pmap")
#println(pi_pmap(100_000_000))
@btime pi_pmap(1_000_000_000);
```

### ClusterManager: Slurm script

```bash
> cat pi_integral_distributed_slurm.sbatch
#!/bin/bash
#SBATCH --time=00:15:00
#SBATCH --nodes=4
#SBATCH --ntasks-per-node=1

module load gcc julia
julia pi_integral_distributed_slurm.jl

> sbatch pi_integral_distributed_slurm.sbatch
Submitted batch job 20831482

> cat slurm-20831482.out
connecting to worker 1 out of 4
connecting to worker 2 out of 4
connecting to worker 3 out of 4
connecting to worker 4 out of 4
Pi pmap
  382.432 ms (343 allocations: 14.52 KiB)
```

### Practical session 3

- Go to `temperatures` folder and modify `temperatures.jl` in order to use the `Distributed` module
- Reserve a node with 4 cores and study the scalability of your code with 1, 2, 4 processes
- Create a Slurm script in order to run your code on 4 nodes/1 core per node ; the code have to be slightly modified to run with Slurm

### MPI (Message Passing Interface)

- Standard interface to develop distributed memory applications in HPC world
  - Communication infrastructure/topology
  - Set of function to perform
    - Point-to-point communications (send, receive, …)
    - Collective communications (broadcast, scatter, gather, all-to-all, reduce, …)
    - One sided communications (remote memory)
    - I/O
- Julia implementation is described at https://juliaparallel.org/MPI.jl/stable (similar to MPI4Py)


### MPI - Hello World


```julia
using MPI
MPI.Init()

comm = MPI.COMM_WORLD
print("Hello world, I am rank $(MPI.Comm_rank(comm)) of $(MPI.Comm_size(comm))\n")
MPI.Barrier(comm)
MPI.Finalize()
```


### MPI - PI computation

```julia
using MPI

function loop(istart, iend, step)
  sum = 0.0
  for i in istart:iend
    sum += 4.0 / (1.0 + ((i + 0.5) * step)^2)
  end
  return sum
end

function pi_mpi(num_steps, rank, size)
  step = 1.0 / num_steps
  chunk_size = num_steps ÷ size
  istart = rank * chunk_size
  iend = istart + (chunk_size - 1)
  r = loop(istart, iend, step)
  return r / num_steps
end

MPI.Init()
const root = 0
const n = 100_000_000
const comm = MPI.COMM_WORLD
const rank = MPI.Comm_rank(comm)
const size = MPI.Comm_size(comm)

r = MPI.Reduce(pi_mpi(n, rank, size), (+), root, comm)

if rank == root
  println("Pi MPI")
  println(r)
end
MPI.Finalize()
```

### MPI on Curnagl - Installation of the package

```bash
module load gcc mvapich2 julia
julia -e 'import Pkg;Pkg.add("MPIPreferences")'
julia -e 'using MPIPreferences;MPIPreferences.use_system_binary();import Pkg;Pkg.add("MPI")'
```


### MPI on Curnagl - Example of Slurm script
```bash
#!/bin/bash

#SBATCH --time 00-00:15:00
#SBATCH --mem=4G
#SBATCH --nodes 4
#SBATCH --ntasks 4

module load gcc mvapich2 julia

#export LD_PRELOAD=$MVAPICH2_ROOT/lib/libmpi.so
export MV2_USE_ALIGNED_ALLOC=1
export JULIA_MPIEXEC=/usr/bin/srun
export JULIA_MPI_BINARY=system

srun julia my_mpi_code.jl
```


## GPU computing


### Simple operation on arrays

```julia
using CUDA

a = CuArray([1, 2, 3, 4, 5])
a .+= 1
```

### PI computation on GPU

```julia
using CUDA
#using BenchmarkTools

function pi(num_steps)
  step = 1.0 / num_steps
  sum = 0.0
  for i in 0:num_steps - 1
    sum += 4.0 / (1.0 + ((i + 0.5) * step)^2)
  end
  return sum / num_steps
end

function pi_gpu(num_steps)
  step = 1.0 / num_steps
  values = CuArray{Float64}(undef, num_steps)
  values[:] .= 4.0 ./ (1.0 .+ (((0:(num_steps - 1)) .+ 0.5) .* step).^2)
  return sum(values) / num_steps
end

#@btime pi(5_120_000_000);
#@btime CUDA.@sync pi_gpu(5_120_000_000);
pi(1_024)
@time pi(5_120_000_000);
CUDA.@sync pi_gpu(1_024)
@CUDA.time pi_gpu(5_120_000_000);

# CPU version
# 15.669320 seconds
# GPU version
#  0.685796 seconds (475.53 k CPU allocations: 25.266 MiB, 3.42% gc time) (3 GPU allocations: 38.147 GiB, 15.55% memmgmt time)
```

### Write your own kernel

```julia
using CUDA
using Test

function vadd!(a, b, c)
    i = (blockIdx().x - 1) * blockDim().x + threadIdx().x
    c[i] = a[i] + b[i]
    return nothing
end

a = round.(rand(10, 10) * 100)
b = round.(rand(10, 10) * 100)
c = similar(a)

d_a = CuArray(a)
d_b = CuArray(b)
d_c = CuArray(c)

@cuda threads=length(a) vadd!(d_a, d_b, d_c)
c = Array(d_c)
@test a+b ≈ c
```

### PI computation on GPU with kernel

```julia
using CUDA

function pi_gpu(num_steps)
  step = 1.0 / num_steps
  values = CuArray{Float64}(undef, num_steps)
  values[:] .= 4.0 ./ (1.0 .+ (((0:(num_steps - 1)) .+ 0.5) .* step).^2)
  return sum(values) / num_steps
end

function kernel!(values, step)
  i = (blockIdx().x - 1) * blockDim().x + threadIdx().x
  values[i] = 4.0 / (1.0 + ((i + 0.5) * step)^2)
  return nothing
end

function pi_gpu_kernel(num_steps)
  step = 1.0 / num_steps
  values = CuArray{Float64}(undef, num_steps)
  threads_per_block = 1024
  blocks = num_steps ÷ threads_per_block
  @cuda threads=threads_per_block blocks=blocks kernel!(values, step)
  return sum(values) / num_steps
end

CUDA.@sync pi_gpu(1_024)
@CUDA.time pi_gpu(5_120_000_000);
CUDA.@sync pi_gpu_kernel(1_024)
@CUDA.time pi_gpu_kernel(5_120_000_000);

# GPU version
#   0.685796 seconds (475.53 k CPU allocations: 25.266 MiB, 3.42% gc time) (3 GPU allocations: 38.147 GiB, 15.55% memmgmt time)
# GPU with kernel version
#  0.081001 seconds (22.80 k CPU allocations: 1.164 MiB) (3 GPU allocations: 38.147 GiB, 0.04% memmgmt time)
```

### Practical session 4: 2D heat propagation

### Practical session 4: 2D heat propagation (part 1)

1. Go to `heat2d` folder and try to execute `heat_propagation.jl`. You can modify the code to see the evolution of the heat propagation.
1. Complete `run_cpu_threads` function to use multi-threading. Then you can study the scalability of the code.

### Practical session 4: 2D heat propagation (part 2)

1. Complete `run_cpu_vect` function using a vector based version so that you can remove the inner loops (`for i in 2:nx-1` and `for j in 2:ny-1`). Why can't you get rid of the external loop (`for m in 1:nsteps`) ?
1. Look at https://docs.julialang.org/en/v1/manual/performance-tips/#man-performance-views to improve the performance of `run_cpu_vect`.

### Practical session 4: 2D heat propagation (part 3)

1. Complete `run_gpu` function to create a CUDA version of the code with `CuArray` and a vector based version of the code.
1. Complete `run_gpu_kernel` and `_gpu_kernel!` functions to create a CUDA kernel version of the code. You can play a bit with the value of `threads_per_block` to see the impact on performance.


## Tips to improve performance


### @fastmath macro

- used to loosen the constraints of IEEE floating point operations in order to achieve greater performance
- similar to `--fast-math` option in GCC compiler
- should not be used when precision is critical


### @fastmath macro - example


In [18]:
using BenchmarkTools

function sum_diff(x)
  n = length(x)
  d = 1/(n-1)
  s = zero(eltype(x))
  s += (x[2] - x[1]) / d
  for i in 2:length(x)-1
    s += (x[i+1] - x[i]) / (2*d)
  end
  s += (x[n] - x[n-1])/d
end

function sum_diff_fast(x)
  n = length(x)
  d = 1/(n-1)
  s = zero(eltype(x))
  @fastmath s += (x[2] - x[1]) / d
  @fastmath for i in 2:n-1
    s += (x[i+1] - x[i]) / (2*d)
  end
  @fastmath s += (x[n] - x[n-1])/d
end

t = rand(100_000)
println("Without fastmath")
@btime sum_diff($t);
println("With fastmath")
@btime sum_diff_fast($t);

Without fastmath
  154.315 μs (0 allocations: 0 bytes)
With fastmath
  26.916 μs (0 allocations: 0 bytes)


### Column-wise storage

<center><img src="images/column_major.svg" width="500" style="border:0"></center>


### Column-wise storage - impact on iterations


In [19]:
using BenchmarkTools

function row_iter(x)
   s = zero(eltype(x))
   for i = 1:size(x, 1)
      for j = 1:size(x, 2)
       s = s + x[i, j] ^ 2
         x[i, j] = s
      end
   end
end

function col_iter(x)
    s = zero(eltype(x))
    for j = 1:size(x, 2)
       for i = 1:size(x, 1)
          s = s + x[i, j] ^ 2
          x[i, j] = s
       end
    end
end

a = rand(10000, 10000);
println("row-wise iterations")
@btime row_iter($a);
println("Column-wise iterations")
@btime col_iter($a);

row-wise iterations
  1.188 s (0 allocations: 0 bytes)
Column-wise iterations
  167.625 ms (0 allocations: 0 bytes)


### Array initialization


In [20]:
using BenchmarkTools
@btime fill(1, 1000, 1000);
@btime Array{Int64}(undef, 1000, 1000);

  662.631 μs (2 allocations: 7.63 MiB)
  6.907 μs (2 allocations: 7.63 MiB)


> ⚠️ the array elements will contain random values, which could be a security issue


### Global variables

In [21]:
using BenchmarkTools

p = 2

const p2 = 2

function pow_array(x)
  s = 0.0
  for y in x
    s = s + y^p
  end
  return s
end

function pow_array_const(x)
  s = 0.0
  for y in x
    s = s + y^p2
  end
  return s
end

t = rand(100000);
println("Global var")
@btime pow_array($t);
println("Const var")
@btime pow_array_const($t);

Global var
  6.124 ms (300000 allocations: 4.58 MiB)
Const var
  655.650 μs (0 allocations: 0 bytes)


## THE END